In [7]:
!pip install wandb --quiet
!pip install swig --quiet
!pip install gym[all] --quiet
!pip install pygame --quiet

In [8]:
import warnings
warnings.filterwarnings("ignore")

In [9]:
## fixing the pygame error

import os
os.environ['SDL_VIDEODRIVER']='dummy'
import pygame
pygame.display.set_mode((640,480))

<Surface(640x480x32 SW)>

In [10]:
import gym
from gym.envs import box2d
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [6]:
gamma = 0.97
learning_rate = 0.1

In [12]:
class Policy_NN(nn.Module):
    def __init__(self,observation_space,action_space,gamma,learning_rate):
        self.observation_space = observation_space
        self.action_space = action_space
        self.gamma = gamma
        self.learning_rate = learning_rate
    #   self.optimizer = optim.Adam(self.parameters(),lr=self.learning_rate)
        self.net = nn.Sequential(
            nn.Linear(in_features= observation_space, out_features = 128),
            nn.ReLU(),
            nn.Linear(in_features= 128, out_features = action_space))

    def forward(self,x):
      return self.net(x) #forward propagation

    def act(self,obs):
        #With obs, find the maximum Q value and output the corresponding action
        obs_tensor = torch.from_numpy(s).unsqueeze(0)
        q_value = self(obs_tensor.unsqueeze(0)) #convert to row vector
        highest_q_value = torch.argmax(input=q_value)
        log_value = torch.log(q_value.squeeze(0)[highest_q_value])
        action = max_q_idx.detach().item() #action corresponding to max Q index
        return action

In [16]:
class reinforce_update(Policy_NN, rewards, log_value):
    discount_rewards = []

    for i in range(len(rewards)):
        Gt = 0
        pw = 0
        for r in rewards[i:]:
            Gt = Gt + gamma**pw * r
            pw = pw + 1
    discount_rewards.append(Gt)

    steps_value = np.arange(rewards.size)

    discount_rewards = torch.tensor(discount_rewards)
    discount_rewards = discount_rewards[::-1].cumsum()[::-1] / gamma**t_steps

    # def discount_rewards(rewards, gamma):
    # t_steps = np.arange(rewards.size)
    # r = rewards * gamma**t_steps
    # r = r[::-1].cumsum()[::-1] / gamma**t_steps
    # return r

    policy_gradient = []
    for log_prob, e_d_r in zip(log_value, discount_rewards):
        policy_gradient.append(-log_prob*e_d_r)

    policy_network.optimizer.zero_grad()
    policy_gradient = torch.stack(policy_gradient).sum()
    policy_gradient.backward()
    policy_network.optimizer.step()

NameError: name 'rewards' is not defined

In [15]:
def main():
    env = gym.make("LunarLander-v2", render_mode="human")
    observation_space = env.observation_space.shape[0]
    action_space = env.action_space.n
    policy_network = Policy_NN(observation_space,action_space,gamma,learning_rate)

    episode_array = []
    rewards_array = []
    Reward_list = np.empty(shape=n_episode)
    n_episode = 1000
    n_timesteps = 1000

    for episode in range(n_episode):
        s = env.reset()
        log_probs = []

        for step in range(n_timesteps):
            action,log_prob = policy_network.action(s)#
            s_next, r, done, _  = env.step(action)
            log_probs.append(log_prob)
            rewards_array.append(r)
            if done:
                reinforce_update(policy_net, rewards_array,log_probs)
                all_rewards.append(np.sum(rewards_array))
        '''Fix Q-target'''
    if episode % TARGET_UPDATE ==0:
        agent.target_net.load_state_dict(agent.online_net.state_dict())
        reward = np.mean(Reward_list[episode-10:episode])
        print("Episode:{}".format(episode))
        print("Reward:{}".format(reward))
        episode_array.append(episode)
        rewards_array.append(reward)
        #set x and y for the final plot

In [ ]:
if __name__ == "__main__":
    main()

AttributeError: 'Policy_NN' object has no attribute '_parameters'

In [ ]:
x = episode_array
y = rewards_array

#plotting the comparison between the two
plt.title("Performance of DQN with and without Experience Replay")
plt.plot(x, y, label = "DQN")
plt.plot(x2, y2, label = "DQN-ER")
plt.xlabel("Episodes")
plt.ylabel("Rewards")
plt.legend()
plt.show()

# Adding WandB section

In [ ]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
import wandb
import random

In [ ]:
# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="my-awesome-project",

    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.02,
    "architecture": "CNN",
    "dataset": "CIFAR-100",
    "epochs": 10,
    }
)

# simulate training
epochs = 10
offset = random.random() / 5
for epoch in range(2, epochs):
    acc = 1 - 2 ** -epoch - random.random() / epoch - offset
    loss = 2 ** -epoch + random.random() / epoch + offset

    # log metrics to wandb
    wandb.log({"acc": acc, "loss": loss})

# [optional] finish the wandb run, necessary in notebooks
wandb.finish()

wandb: Currently logged in as: itismeganrms (universiteitleiden). Use `wandb login --relogin` to force relogin


acc,▁▅▄█▆█▆█
loss,█▆▂▃▃▂▁▁
acc,0.79491
loss,0.25435
